# Comparing Retrieval Augmented Generation (RAG) Methods with Graphs

In this notebook we will explore and compare several RAG methods with a focus on knowledge graphs.

## Background on RAG:

Below digrams show how RAG works:

```
                  RAG with Llama Index
                  ┌────┬────┬────┬────┐                  
                  │ 1  │ 2  │ 3  │ 4  │                  
                  ├────┴────┴────┴────┤                  
                  │  Docs/Knowledge   │                  
┌───────┐         │        ...        │       ┌─────────┐
│       │         ├────┬────┬────┬────┤       │         │
│       │         │ 95 │ 96 │    │    │       │         │
│       │         └────┴────┴────┴────┘       │         │
│ User  │─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─▶   LLM   │
│       │                                     │         │
│       │                                     │         │
└───────┘    ┌ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ┐  └─────────┘
    │          ┌──────────────────────────┐        ▲     
    └────────┼▶│  Tell me ....., please   │├───────┘     
               └──────────────────────────┘              
             │ ┌────┐ ┌────┐               │             
               │ 3  │ │ 96 │                             
             │ └────┘ └────┘               │             
              ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
```

In VectorDB based RAG, we create embeddings of each node (chunk), and find TopK related ones towards a given question during the query. In the above diagram, nodes `3` and `96` were fetched as the TopK related nodes and used to answer the user query. 

## Background Graph RAG

In Graph RAG, we will extract relationships between representing concise facts from each node. It would look something like this:

```
Node Split and Embedding

┌────┬────┬────┬────┐
│ 1  │ 2  │ 3  │ 4  │
├────┴────┴────┴────┤
│  Docs/Knowledge   │
│        ...        │
├────┬────┬────┬────┤
│ 95 │ 96 │    │    │
└────┴────┴────┴────┘
```

If we zoomed in:

```
       Node Split and Embedding, with Knowledge Graph being extracted

┌──────────────────┬──────────────────┬──────────────────┬──────────────────┐
│ .─.       .─.    │  .─.       .─.   │            .─.   │  .─.       .─.   │
│( x )─────▶ y )   │ ( x )─────▶ a )  │           ( j )  │ ( m )◀────( x )  │
│ `▲'       `─'    │  `─'       `─'   │            `─'   │  `─'       `─'   │
│  │     1         │        2         │        3    │    │        4         │
│ .─.              │                  │            .▼.   │                  │
│( z )─────────────┼──────────────────┼──────────▶( i )─┐│                  │
│ `◀────┐          │                  │            `─'  ││                  │
├───────┼──────────┴──────────────────┴─────────────────┼┴──────────────────┤
│       │                      Docs/Knowledge           │                   │
│       │                            ...                │                   │
│       │                                               │                   │
├───────┼──────────┬──────────────────┬─────────────────┼┬──────────────────┤
│  .─.  └──────.   │  .─.             │                 ││  .─.             │
│ ( x ◀─────( b )  │ ( x )            │                 └┼▶( n )            │
│  `─'       `─'   │  `─'             │                  │  `─'             │
│        95   │    │   │    96        │                  │   │    98        │
│            .▼.   │  .▼.             │                  │   ▼              │
│           ( c )  │ ( d )            │                  │  .─.             │
│            `─'   │  `─'             │                  │ ( x )            │
└──────────────────┴──────────────────┴──────────────────┴──`─'─────────────┘
```

In theory, knowledge graphs should help balance granularity and density. Optionally, multi-hop of `x -> y`, `i -> j -> z -> x` etc... across many more nodes (chunks) than TopK search allows. 

# 1. Preparation

## 1.1 Prepare for LLM

In [1]:
import os
from dotenv import load_dotenv

import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import (
    KnowledgeGraphIndex,
    ServiceContext,
    VectorStoreIndex,
    SimpleDirectoryReader,
    LLMPredictor,
)
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import SimpleGraphStore


from IPython.display import Markdown, display, HTML

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


In [2]:
# For Azure OpenAI
# import os
# import json
# import openai
# from langchain.embeddings import OpenAIEmbeddings
# from llama_index.llms import AzureOpenAI
# from llama_index import LangchainEmbedding
from llama_index import (
    VectorStoreIndex,
    KnowledgeGraphIndex,
)
from llama_index import set_global_service_context

from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores.simple import SimpleGraphStore

# openai.api_type = "azure"
# openai.api_base = "https://<foo-bar>.openai.azure.com"
# openai.api_version = "2022-12-01"
# os.environ["OPENAI_API_KEY"] = "youcannottellanyone"
# openai.api_key = os.getenv("OPENAI_API_KEY")

# llm = AzureOpenAI(
#     engine="<foo-bar-deployment>",
#     temperature=0,
#     openai_api_version=openai.api_version,
#     model_kwargs={
#         "api_key": openai.api_key,
#         "api_base": openai.api_base,
#         "api_type": openai.api_type,
#         "api_version": openai.api_version,
#     },
# )

# You need to deploy your own embedding model as well as your own chat completion model
# embedding_llm = LangchainEmbedding(
#     OpenAIEmbeddings(
#         model="text-embedding-ada-002",
#         deployment="<foo-bar-deployment>",
#         openai_api_key=openai.api_key,
#         openai_api_base=openai.api_base,
#         openai_api_type=openai.api_type,
#         openai_api_version=openai.api_version,
#     ),
#     embed_batch_size=1,
# )

# embedding_llm = LangchainEmbedding(
#     OpenAIEmbeddings())

# service_context = ServiceContext.from_defaults(
#     llm=llm,
#     embed_model=embedding_llm,
# )

# set_global_service_context(service_context)

## 1.2. Prepare  Graph Store


In [3]:
graph_store = SimpleGraphStore()

storage_context = StorageContext.from_defaults(graph_store=graph_store)

## 2. Build the Knowledge Graph

The Knowledge Graph is created with the `KnowledgeGraphIndex` from Llama Index. This extracts "Triplets" that will be persisted in `SimpleGraphStore`.

### 2.1 Preprocess Data

We will download and preprecess data from:
    https://www.iii.org/sites/default/files/docs/pdf/HO3_sample.pdf

In [4]:
from llama_index import SimpleDirectoryReader
from ..src_index.clean_sample_ho3 import clean_sample_ho3_pages

documents = SimpleDirectoryReader(input_files=['../data/HO3_sample.pdf']).load_data()

for i, _ in enumerate(documents):
    documents[i].text = clean_sample_ho3_pages(documents[i].text)
    documents[i].text = documents[0].text.lower()

ImportError: attempted relative import with no known parent package

* PDF page count for the HO3 Homeowner's policy document

In [ ]:
len(documents)

22

* Prompt to build the graph:

In [ ]:
from llama_index.prompts.base import Prompt
from llama_index.prompts.prompt_type import PromptType


KG_TRIPLET_EXTRACT_TMPL = (
    "Some text is provided below. Given the text, extract up to "
    "{max_knowledge_triplets} "
    "knowledge triplets in the form of (subject, predicate, object). Avoid stopwords, page_label, and numbers.\n"
    "---------------------\n"
    "Example:"
    "Text: Alice is Bob's mother."
    "Triplets:\n(Alice, is mother of, Bob)\n"
    "Text: Philz is a coffee shop founded in Berkeley in 1982.\n"
    "Triplets:\n"
    "(Philz, is, coffee shop)\n"
    "(Philz, founded in, Berkeley)\n"
    "(Philz, founded in, 1982)\n"
    "---------------------\n"
    "Text: {text}\n"
    "Triplets:\n"
)
KG_TRIPLET_EXTRACT_PROMPT = Prompt(
    KG_TRIPLET_EXTRACT_TMPL, prompt_type=PromptType.KNOWLEDGE_TRIPLET_EXTRACT
)

### 2.2 Extract Triplets and Save to SimpleGraphStore

This call will take some time, it'll extract entities and relationships and store them in SimpleGraphStore

In [ ]:
from llama_index.llms import OpenAI
from llama_index import OpenAIEmbedding, ServiceContext, KnowledgeGraphIndex

embedding_llm = OpenAIEmbedding()

llm=OpenAI(temperature=0, 
           model_name="gpt-3.5-turbo",
           )

service_context = ServiceContext.from_defaults(llm=llm, 
                                               embed_model=embedding_llm,
                                               chunk_size=512,
                                               chunk_overlap=100
                                               )

set_global_service_context(service_context)

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=10,
    storage_context=storage_context,
    include_embeddings=True,
    kg_triple_extract_template=KG_TRIPLET_EXTRACT_PROMPT,
    show_progress=True)

Processing nodes:  89%|████████▉ | 59/66 [03:16<00:28,  4.06s/it]

INFO:openai:error_code=None error_message='Request failed due to server shutdown' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
error_code=None error_message='Request failed due to server shutdown' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
error_code=None error_message='Request failed due to server shutdown' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
  "error": {
    "message": "Request failed due to server shutdown",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'Request failed due to server shutdown', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Mon, 24 Jul 2023 05:33:27 GMT', 'Content-Type': 'application/json', 'Content-Length': '141', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'openai-model': 'text-davinci-003', 'openai-organization': 'us

Processing nodes: 100%|██████████| 66/66 [03:44<00:00,  3.40s/it]


## 3 Create VectorStoreIndex for RAG

To compare Graph-based query with vector similarity we'll also create a `VectorStoreIndex`.

During the creation, the same data source will be split into chunks and converted to embeddings. During query time, the top-k related embeddings will be vector-searched with the embedding of the question.

```
                  RAG with Llama Index
                  ┌────┬────┬────┬────┐                  
                  │ 1  │ 2  │ 3  │ 4  │                  
                  ├────┴────┴────┴────┤                  
                  │  Docs/Knowledge   │                  
┌───────┐         │        ...        │       ┌─────────┐
│       │         ├────┬────┬────┬────┤       │         │
│       │         │ 95 │ 96 │    │    │       │         │
│       │         └────┴────┴────┴────┘       │         │
│ User  │─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─▶   LLM   │
│       │                                     │         │
│       │                                     │         │
└───────┘    ┌ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ┐  └─────────┘
    │          ┌──────────────────────────┐        ▲     
    └────────┼▶│  Tell me ....., please   │├───────┘     
               └──────────────────────────┘              
             │ ┌────┐ ┌────┐               │             
               │ 3  │ │ 96 │                             
             │ └────┘ └────┘               │             
              ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
```

The next line of code does this:

In [ ]:
from llama_index import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(
    documents,
)

In [ ]:
# persist KG Index
kg_index.set_index_id = "kg_ho3_policy"
kg_index.storage_context.persist(persist_dir='./ho3_storage_kg_graph')

# persist Vector Index
vector_index.set_index_id = "vector_ho3_policy"
vector_index.storage_context.persist(persist_dir='./ho3_storage_kg_vector')

NameError: name 'kg_index' is not defined

## 4. Persist and Load index objects from disk

Both the `KnowledgeGraphIndex` and `VectorStoreIndex` will be created only once. Afterwards, we will persist and re-use them. Adding IDs is optional and only needed when multiple index objects exist in the same directory.

#### Persist

```python
# persist KG Index
kg_index.set_index_id = "kg_ho3_policy"
kg_index.storage_context.persist(persist_dir='./storage_graph')

# persist Vector Index
vector_index.set_index_id = "vector_ho3_policy"
vector_index.storage_context.persist(persist_dir='./storage_vector')

```
#### Restore

Restore the index from disk like:

```python
from llama_index import load_index_from_storage

# Set ServiceContext if it hasn't already been done
embedding_llm = OpenAIEmbedding()
llm=OpenAI(temperature=0, 
           model_name="gpt-3.5-turbo",
)
service_context = ServiceContext.from_defaults(llm=llm, 
                                               embed_model=embedding_llm,
                                               chunk_size=512,
)

storage_context = StorageContext.from_defaults(persist_dir='./storage_graph', graph_store=graph_store)
kg_index = load_index_from_storage(
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    include_embeddings=True,
)

storage_context_vector = StorageContext.from_defaults(persist_dir='./storage_vector')
vector_index = load_index_from_storage(
    service_context=service_context,
    storage_context=storage_context_vector
)
```

In [ ]:
from llama_index.llms import OpenAI
from llama_index import (
    OpenAIEmbedding, 
    ServiceContext, 
    load_index_from_storage, 
    set_global_service_context
)


embedding_llm = OpenAIEmbedding()

llm=OpenAI(temperature=0, 
           model_name="gpt-3.5-turbo",
           )

service_context = ServiceContext.from_defaults(llm=llm, 
                                               embed_model=embedding_llm,
                                               chunk_size=512,
                                               )

set_global_service_context(service_context)

storage_context_kg = StorageContext.from_defaults(graph_store=graph_store, persist_dir="./ho3_storage_kg_graph")
kg_index = load_index_from_storage(
    storage_context=storage_context_kg,
    max_triplets_per_chunk=10,
    include_embeddings=True, 
    )

storage_context_vector = StorageContext.from_defaults(persist_dir='./ho3_storage_kg_vector')
vector_index = load_index_from_storage(
    storage_context=storage_context_vector,
)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.


Loading all indices.
INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.


## 5. Prepare for different query approaches

We will do 4 types of query approaches with LLM, KG, VectorDB:

| QueryEngine | Knowledge Graph Only                                 | Graph RAG query engine                                       | Vector RAG query engine                                      | Graph Vector RAG query engine                                |
| ----------- | ------------------------------------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ |
| Mechanism   | 1. **Graph Only** based on triplets<br />2. Query KG with the result<br />3. Answer synthesis based on query result | 1. Get related entities of the question<br />2. Get n-depth **SubGraphs** of related entities from KG<br />3. Answer synthesis based on related SubGraphs | 1. Create embedding of question<br />2. Semantic search **top-k related doc chunks**<br />3. Answer synthesis based on related doc chunks | 1. Do retrieval as Vector and Graph RAG <br />2. Answer synthesis based on **both related chunks and SubGraphs** |


### 5.1 Knowledge Graph Only

This approach strictly follows the graph.

In [ ]:
kg_query_engine = kg_index.as_query_engine(
    # Uses the raw triplets instead of adding the text from the corresponding nodes
    include_text=True,
    retriever_mode="keyword",
    response_mode="tree_summarize",
)

### 5.1.1 Knowledge Graph Hybrid - Search 

Here, we add flexibility from embedding representations.

In [ ]:
# query using top 3 triplets plus keywords (duplicate triplets are removed)
hybrid_query_engine = kg_index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
)

### 5.2 Graph RAG query engine

Graph RAG takes SubGraphs related to entities of the task/question as Context.

```
           Graph + Vector RAG with Llama Index
                  ┌────┬────┬────┬────┐                  
                  │ 1  │ 2  │ 3  │ 4  │                  
                  ├────┴────┴────┴────┤                  
                  │  Docs/Knowledge   │                  
┌───────┐         │        ...        │       ┌─────────┐
│       │         ├────┬────┬────┬────┤       │         │
│       │         │ 95 │ 96 │    │    │       │         │
│       │         └────┴────┴────┴────┘       │         │
│ User  │─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─▶   LLM   │
│       │                                     │         │
│       │                                     │         │
└───────┘    ┌ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ┐  └─────────┘
    │          ┌──────────────────────────┐        ▲     
    └────────┼▶│  Tell me about x, please │├───────┘     
               └──────────────────────────┘              
             │ Below are knowledge about x │             
               x->y<-z,x->h->i, m<-n,...                            
             │ Please answer based on them │             
              ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
```

In [ ]:
kg_rag_query_engine = kg_index.as_query_engine(
    include_text=False,
    retriever_mode="keyword",
    response_mode="tree_summarize",
)

### 5.3 Standard vector query engine

Vector RAG to find topK semantic related doc chunks, and use as context for the answer.

In [ ]:
vector_rag_query_engine = vector_index.as_query_engine()

### 5.4 Graph+Vector RAG query engine

This is a combined Graph+Vector Based RAG, where we will retrieve both VectorDB and KG SubGraphs as the context for the answer.

```
           Graph + Vector RAG with Llama Index
                  ┌────┬────┬────┬────┐                  
                  │ 1  │ 2  │ 3  │ 4  │                  
                  ├────┴────┴────┴────┤                  
                  │  Docs/Knowledge   │                  
┌───────┐         │        ...        │       ┌─────────┐
│       │         ├────┬────┬────┬────┤       │         │
│       │         │ 95 │ 96 │    │    │       │         │
│       │         └────┴────┴────┴────┘       │         │
│ User  │─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─▶   LLM   │
│       │                                     │         │
│       │                                     │         │
└───────┘    ┌ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ┐  └─────────┘
    │          ┌──────────────────────────┐        ▲     
    └────────┼▶│  Tell me ....., please   │├───────┘     
               └──────────────────────────┘              
             │ ┌────┐┌────┐               │             
               │ 3  ││ 96 │ x->y<-z,x->h...                            
             │ └────┘└────┘               │             
              ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
```

To implement that in Llama Index, we create a `CustomRetriever` to comebine the two: 

In [ ]:
# import QueryBundle
from llama_index import QueryBundle

# import NodeWithScore
from llama_index.schema import NodeWithScore

# Retrievers
from llama_index.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever

from typing import List


class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both Vector search and Knowledge Graph search"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        kg_retriever: KGTableRetriever,
        mode: str = "OR",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        kg_nodes = self._kg_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(kg_ids)
        else:
            retrieve_ids = vector_ids.union(kg_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

Next, we will create instances of the Vector and KG retrievers, which will be used in the instantiation of the Custom Retriever.

In [ ]:
from llama_index import get_response_synthesizer
from llama_index.query_engine import RetrieverQueryEngine

# create custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index)

kg_retriever = KGTableRetriever(
    index=kg_index, retriever_mode="keyword", include_text=False
)
custom_retriever = CustomRetriever(vector_retriever, kg_retriever)

# create response synthesizer
response_synthesizer = get_response_synthesizer(
    service_context=service_context,
    response_mode="tree_summarize",
)

And the query engine:

In [ ]:
graph_vector_rag_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

## 6. Query with all the Engines
* We will ask a purposefully vague and open-ended query

### 6.1 Keyword Knowledge Graph Query

In [ ]:
response_kg = kg_query_engine.query("Tell me about the limit of liability.")

display(Markdown(f"<b>{response_kg}</b>"))

INFO:llama_index.indices.knowledge_graph.retriever:> Starting query: Tell me about the limit of liability.
> Starting query: Tell me about the limit of liability.
> Starting query: Tell me about the limit of liability.
INFO:llama_index.indices.knowledge_graph.retriever:> Query keywords: ['Limit', 'Responsibility', 'Liability']
> Query keywords: ['Limit', 'Responsibility', 'Liability']
> Query keywords: ['Limit', 'Responsibility', 'Liability']
INFO:llama_index.indices.knowledge_graph.retriever:> Extracted relationships: The following are knowledge triplets in max depth 2 in the form of `subject [predicate, object, predicate_next_hop, object_next_hop ...]`
> Extracted relationships: The following are knowledge triplets in max depth 2 in the form of `subject [predicate, object, predicate_next_hop, object_next_hop ...]`
> Extracted relationships: The following are knowledge triplets in max depth 2 in the form of `subject [predicate, object, predicate_next_hop, object_next_hop ...]`


<b>
The limit of liability is the maximum amount of money that a person or company is legally responsible for paying in the event of a lawsuit or other legal action. It is typically specified in a contract or insurance policy and is intended to protect the party from excessive financial losses.</b>

### 6.2 Graph Keyword Embeddings Hybrid

In [ ]:
response_hybrid = hybrid_query_engine.query("Tell me about the limit of liability.")

display(Markdown(f"<b>{response_hybrid}</b>"))

INFO:llama_index.indices.knowledge_graph.retriever:> Starting query: Tell me about the limit of liability.
> Starting query: Tell me about the limit of liability.
> Starting query: Tell me about the limit of liability.
INFO:llama_index.indices.knowledge_graph.retriever:> Query keywords: ['Limit', 'Responsibility', 'Liability']
> Query keywords: ['Limit', 'Responsibility', 'Liability']
> Query keywords: ['Limit', 'Responsibility', 'Liability']
INFO:llama_index.indices.knowledge_graph.retriever:> Querying with idx: a7221853-4e5b-402f-ba1a-1384b2ce7496: homeowners
ho 00 03 10 00
ho 00 03 10 00 copyright, insurance services office...
> Querying with idx: a7221853-4e5b-402f-ba1a-1384b2ce7496: homeowners
ho 00 03 10 00
ho 00 03 10 00 copyright, insurance services office...
> Querying with idx: a7221853-4e5b-402f-ba1a-1384b2ce7496: homeowners
ho 00 03 10 00
ho 00 03 10 00 copyright, insurance services office...
INFO:llama_index.indices.knowledge_graph.retriever:> Querying with idx: f880f431-4

<b>
The limit of liability for motor vehicle and watercraft liability is the amount of coverage provided for bodily injury. This coverage is subject to the provisions of the policy and may vary depending on the type of vehicle or craft.</b>

### 6.3 Vector RAG

In [ ]:
response_vector_rag = vector_rag_query_engine.query("Tell me about the limit of liability")

display(Markdown(f"<b>{response_vector_rag}</b>"))

<b>
The limit of liability in this policy is the maximum amount of coverage that is provided for any claims arising from the ownership, maintenance, occupancy, operation, use, loading or unloading of an aircraft, hovercraft, motor vehicle, or watercraft by an insured. This limit of liability is subject to the provisions outlined in the policy.</b>

### 6.4 Graph + Vector RAG

In [ ]:
response_graph_vector_rag = graph_vector_rag_query_engine.query("Tell me about the limit of liability.")

Markdown(f"<b>{response_graph_vector_rag}</b>")

INFO:llama_index.indices.knowledge_graph.retriever:> Starting query: Tell me about the limit of liability.
> Starting query: Tell me about the limit of liability.
> Starting query: Tell me about the limit of liability.
INFO:llama_index.indices.knowledge_graph.retriever:> Query keywords: ['Limit', 'Responsibility', 'Liability']
> Query keywords: ['Limit', 'Responsibility', 'Liability']
> Query keywords: ['Limit', 'Responsibility', 'Liability']
INFO:llama_index.indices.knowledge_graph.retriever:> Extracted relationships: The following are knowledge triplets in max depth 2 in the form of `subject [predicate, object, predicate_next_hop, object_next_hop ...]`
> Extracted relationships: The following are knowledge triplets in max depth 2 in the form of `subject [predicate, object, predicate_next_hop, object_next_hop ...]`
> Extracted relationships: The following are knowledge triplets in max depth 2 in the form of `subject [predicate, object, predicate_next_hop, object_next_hop ...]`


<b>
The limit of liability is defined in the policy as the maximum amount of money that the company providing the insurance will pay out in the event of a claim. This amount is typically determined by the type of coverage purchased and the amount of the premium paid. The policy may also specify certain exclusions or limitations on the coverage, such as the types of losses that are not covered or the maximum amount of liability for certain types of losses.</b>

## 7. Summary

### 7.1 Overall Comparision

In [ ]:
# Set up a new model for QA
from langchain.llms import OpenAI

qa_llm = OpenAI(temperature=0)

* Asking GPT-3.5-Turbo to compare query results

In [ ]:

Markdown(
    qa_llm(f"""
Compare the QA results on "Explain Limit of Liability", list the knowledge facts between them to help evalute them. Output in markdown table.

Result GraphQuery: {response_kg}
---
Result GraphHybrid: {response_hybrid}
---
Result Vector: {response_vector_rag}
---
Result Graph+Vector: {response_graph_vector_rag}
---

"""
    )
)


| Knowledge Facts | GraphQuery | GraphHybrid | Vector | Graph+Vector |
| --- | --- | --- | --- | --- |
| Definition | The maximum amount of money that a person or company is legally responsible for paying in the event of a lawsuit or other legal action. | The amount of coverage provided for bodily injury. | The maximum amount of coverage that is provided for any claims arising from the ownership, maintenance, occupancy, operation, use, loading or unloading of an aircraft, hovercraft, motor vehicle, or watercraft by an insured. | The maximum amount of money that the company providing the insurance will pay out in the event of a claim. |
| Specified in | Contract or insurance policy | Policy | Policy | Policy |
| Intended to | Protect the party from excessive financial losses | - | - | - |
| Coverage varies | - | Depending on the type of vehicle or craft | - | Depending on the type of coverage purchased and the amount of the premium paid |
| Exclusions/Limitations | - | - | Outlined in the policy | Types of losses that are not covered or the maximum amount of liability for certain types of losses |

In [ ]:

Markdown(
    qa_llm(f"""
Compare the two QA result on "Explain Limit of Liability", list the differences between them to help evalute them. Output in markdown table.

Result from Graph: {response_kg}
---
Result from Graph_hybrid: {response_hybrid}

"""     )
    )




| Result from Graph | Result from Graph_hybrid |
| --- | --- |
| The limit of liability is the maximum amount of money that a person or company is legally responsible for paying in the event of a lawsuit or other legal action. It is typically specified in a contract or insurance policy and is intended to protect the party from excessive financial losses. | The limit of liability for motor vehicle and watercraft liability is the amount of coverage provided for bodily injury. This coverage is subject to the provisions of the policy and may vary depending on the type of vehicle or craft. |

In [ ]:

Markdown(
    qa_llm(f"""
Compare the two QA results on "Explain Limit of Liability", list the differences between them, to help evalute them. Output in markdown table.

Result from Topk Vecs: {response_vector_rag}
---
Result from Graph+Vec: {response_graph_vector_rag}

"""     )
    )



| Result | Topk Vecs | Graph+Vec |
| --- | --- | --- |
| Definition | Maximum amount of coverage provided for any claims arising from the ownership, maintenance, occupancy, operation, use, loading or unloading of an aircraft, hovercraft, motor vehicle, or watercraft by an insured. | Maximum amount of money that the company providing the insurance will pay out in the event of a claim. |
| Factors | Subject to the provisions outlined in the policy. | Type of coverage purchased and the amount of the premium paid. The policy may also specify certain exclusions or limitations on the coverage. |

In [ ]:
# Generate and export network graph

from pyvis.network import Network

g = kg_index.get_networkx_graph()
net = Network(
    notebook=True,
    directed=True,
    cdn_resources='in_line',
)

net = Network(
    cdn_resources='local',
    directed = True,            # directed graph
    bgcolor = "#222222",          # background color of graph 
    font_color = "white",      # use yellow for node labels
    height = "1000px",          # height of chart
    width = "100%",             # fill the entire width    
    )

net.repulsion(
    node_distance=200,
    central_gravity=0.2,
    spring_length=250,
    spring_strength=0.08,
    damping=0.1,
)
net.toggle_physics(True)
net.from_nx(g)
net.write_html("kg_index_ho3.html")